In [1]:
from tensorflow import keras
datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1 / 255.0,
                                                       vertical_flip = True,
                                                       horizontal_flip = True)

In [10]:
from keras.applications import DenseNet121

weights_path = r'C:\Users\vidha\Health Analytics\brucechou1983_CheXNet_Keras_0.3.0_weights.h5\brucechou1983_CheXNet_Keras_0.3.0_weights.h5'

conv_base = DenseNet121(weights = None,
include_top = False,
input_shape = (224, 224, 3))

for layer in conv_base.layers:
  layer.trainable = False

predictions = keras.layers.Dense(14, activation='sigmoid', name='predictions')(conv_base.output)

conv_base = keras.Model(inputs = conv_base.input, outputs = predictions)

conv_base.load_weights(weights_path)
        
conv_base.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d_2 (ZeroPadding2  (None, 230, 230, 3)  0          ['input_2[0][0]']                
 D)                                                                                               
                                                                                                  
 conv1/conv (Conv2D)            (None, 112, 112, 64  9408        ['zero_padding2d_2[0][0]']       
                                )                                                           

In [11]:
trainGen = datagen.flow_from_directory(
    r'C:\Users\vidha\Health Analytics\archive\Covid19-dataset\train',
    target_size = (224,224),
    batch_size = 32,
    shuffle = True,
    class_mode = 'categorical'
)

datatestgen = keras.preprocessing.image.ImageDataGenerator(rescale=1 / 255.0)
                                                       
testGen = datatestgen.flow_from_directory(
    r'C:\Users\vidha\Health Analytics\archive\Covid19-dataset\test',
    target_size = (224,224),
    batch_size = 32,
    class_mode = 'categorical'
)

Found 251 images belonging to 3 classes.
Found 66 images belonging to 3 classes.


In [4]:
import numpy as np
def extractfeatures(sample_count, generator):
    i = 0
    batch_size = 32
    features = np.zeros(shape=(sample_count, 7, 7, 14))
    labels = np.zeros(shape=(sample_count, 3))
    for inputs_batch, labels_batch in generator:
        features_batch = conv_base.predict(inputs_batch)
        features[i * batch_size: (i+1) * batch_size] = features_batch
        labels[i * batch_size: (i+1) * batch_size] = labels_batch
        i += 1
        if ((i * batch_size % 1000) == 0 ):
            print("processed size =", i * batch_size)
        if i * batch_size >= sample_count:
            break
    return features, labels

In [5]:
train_features, train_labels = extractfeatures(251, trainGen)
test_features, test_labels = extractfeatures(66, testGen)

1/1 [==============================] - 0s 257ms/step


In [6]:
classifier = keras.Sequential([
    keras.layers.Flatten(input_shape=(7, 7, 14)),
    keras.layers.Dense(256, activation = 'relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(3, activation = 'softmax'),
])
classifier.compile(loss = 'categorical_crossentropy', metrics = ['accuracy'], optimizer = 'adam')

In [7]:
classifier.fit(train_features, train_labels, steps_per_epoch = 251 // 32, epochs = 20, validation_split = 0.2)

Epoch 1/20
7/7 [==============================] - 2s 68ms/step - loss: 1.0918 - accuracy: 0.3900 - val_loss: 1.0621 - val_accuracy: 0.5098
Epoch 2/20
7/7 [==============================] - 0s 11ms/step - loss: 1.0635 - accuracy: 0.4400 - val_loss: 1.0353 - val_accuracy: 0.4902
Epoch 3/20
7/7 [==============================] - 0s 11ms/step - loss: 1.0434 - accuracy: 0.4500 - val_loss: 1.0127 - val_accuracy: 0.5098
Epoch 4/20
7/7 [==============================] - 0s 12ms/step - loss: 1.0282 - accuracy: 0.4450 - val_loss: 0.9912 - val_accuracy: 0.5098
Epoch 5/20
7/7 [==============================] - 0s 12ms/step - loss: 1.0130 - accuracy: 0.4600 - val_loss: 0.9727 - val_accuracy: 0.5098
Epoch 6/20
7/7 [==============================] - 0s 15ms/step - loss: 1.0006 - accuracy: 0.4700 - val_loss: 0.9529 - val_accuracy: 0.5294
Epoch 7/20
7/7 [==============================] - 0s 12ms/step - loss: 0.9779 - accuracy: 0.4800 - val_loss: 0.9377 - val_accuracy: 0.5490
Epoch 8/20
7/7 [===========

In [8]:
classifier.save('CheXNetFinalProject.model')

INFO:tensorflow:Assets written to: CheXNetFinalProject.model\assets


INFO:tensorflow:Assets written to: CheXNetFinalProject.model\assets


In [9]:
classifier.evaluate(test_features, test_labels, batch_size = 32)

3/3 [==============================] - 0s 4ms/step - loss: 0.8227 - accuracy: 0.6970


[0.8227260708808899, 0.6969696879386902]

In [12]:
classifier.evaluate(train_features, train_labels)

8/8 [==============================] - 0s 4ms/step - loss: 0.7110 - accuracy: 0.8088


[0.7109622955322266, 0.8087649345397949]